Simple word2vec model with 200 features for each utterance, not considering other features (speaker etc.)

Doc2Vec model for all chats

In [3]:
import os
import json
import gensim

# path = r"C:\Users\user\git\NLPProj\Frames-dataset\chats"


def build_label_dict():
    d = {}
    label_path = r"/home/amirgilad/Documents/git/NLPProj/Frames-dataset/labels.txt"
    label_file = open(label_path, encoding='utf-8')
    for line in label_file:
        k, v = line.split("\t")
        d[k] = v.strip()
    return d

def gen_chat_model(label_dict):
    chat_path = r"/home/amirgilad/Documents/git/NLPProj/Frames-dataset/chats"
    chats = []
    avg_chat_len = 0
    for filename in os.listdir(chat_path):
        chat_file = open(os.path.join(chat_path, filename), encoding='utf-8')
        chat = json.load(chat_file)
        sents = [] # chat level docs
        if 'turns' in chat:
            for turn in chat['turns']:
                sents.extend( turn['text'].split() )
                avg_chat_len += 1
        tagged_chat = gensim.models.doc2vec.TaggedDocument(sents, label_dict[filename])
        chats.append( tagged_chat )
    avg_chat_len /= len(chats)
    print("average number of sents in chat: " + str(avg_chat_len))
    return gensim.models.Doc2Vec(chats, size=100, window=8, min_count=1, workers=4)

# d = build_label_dict()
# chat_model = gen_chat_model(d)

Prepare input vectors from first parts of chats and labels from dictionary

In [4]:
def infer_vectors(chat_model, label_dict, num_of_sents_from_start):
    '''infer vectors for every first part in the chats. 
    Will be used to load the training data and tags'''
    chat_path = r"/home/amirgilad/Documents/git/NLPProj/Frames-dataset/chats"
    chats_vecs = []
    labels = []
    for filename in os.listdir(chat_path):
        chat_file = open(os.path.join(chat_path, filename), encoding='utf-8')
        chat = json.load(chat_file)
        sents = [] # chat level docs
        if 'turns' in chat:
            lim = min(num_of_sents_from_start, len(chat['turns']))
            for i in range( lim ):
                sents.extend( chat['turns'][i]['text'].split() )
#             for turn in chat['turns']:
#                 sents.extend( turn['text'].split() )
        sent_vec = chat_model.infer_vector( sents )
        chats_vecs.append( sent_vec )
        labels.append(label_dict[filename])
    return np.array(chats_vecs), np.array(labels)

Prepare input vectors for all sents in the first parts of chats and labels from dictionary

In [24]:
def infer_vectors_every_sentence(chat_model, label_dict, num_of_sents_from_start):
    '''infer vectors for every first part in the chats. 
    Will be used to load the training data and tags'''
    chat_path = r"/home/amirgilad/Documents/git/NLPProj/Frames-dataset/chats"
    chats_vecs = []
    labels = []
    for filename in os.listdir(chat_path):
        chat_file = open(os.path.join(chat_path, filename), encoding='utf-8')
        chat = json.load(chat_file)
        sents = [] # chat level docs
        if 'turns' in chat:
            lim = min(num_of_sents_from_start, len(chat['turns']))
            for i in range( lim ):
                sents.append( chat_model.infer_vector([chat['turns'][i]['author']] + chat['turns'][i]['text'].split()) )
            if lim < num_of_sents_from_start:
                diff = num_of_sents_from_start - lim
                lst = [chat_model.infer_vector(chat['turns'][-1]['text'].split()) for i in range(diff)]
                sents.extend( lst )
        chats_vecs.append( sents )
        labels.append(label_dict[filename])
    return np.array(chats_vecs), np.array(labels)

One-hot model for words

In [ ]:
def gen_one_hot_dict():
    chat_path = r"/home/amirgilad/Documents/git/NLPProj/Frames-dataset/chats"
    index_dict = {}
    i = 0
    for filename in os.listdir(chat_path):
        chat_file = open(os.path.join(chat_path, filename), encoding='utf-8')
        chat = json.load(chat_file)
        if 'turns' in chat:
            for turn in chat['turns']:
                for word in turn['text'].split():
                    if word.strip() not in index_dict:
                        index_dict[word.strip()] = i
                        i += 1
    return index_dict

def infer_vectors_from_one_hot(index_dict, label_dict, num_of_sents_from_start):
    '''infer vectors for every sentence in the chats. 
    Will be used to load the training data and tags'''
    chat_path = r"/home/amirgilad/Documents/git/NLPProj/Frames-dataset/chats"
    chats_vecs = []
    for filename in os.listdir(chat_path):
        chat_file = open(os.path.join(chat_path, filename), encoding='utf-8')
        chat = json.load(chat_file)
        sents = [] # chat level docs
        if 'turns' in chat:
            lim = min(num_of_sents_from_start, len(chat['turns']))
            for i in range( lim ):
                sents.extend( [index_dict[word.strip()] for word in chat['turns'][i]['text'].split()] )
                
        sent_vec = 
        chats_vecs.append( sent_vec )
        labels.append(label_dict[filename])
    return np.array(chats_vecs), np.array(labels)

Simple LSTM network - does not perform well

In [199]:
import numpy as np
from keras.layers import Input, Dense, merge, Embedding, RNN
from keras.models import Model
import keras
import sys
# should change to relative import
sys.path.insert(0, '/home/amirgilad/Documents/git/NLPProj/kerasAttention')
import attention_dense
import attention_lstm

input_dims = x_train.shape
train_size = 1000

# Build doc2Vec model for the chats
d = build_label_dict()
chat_model = gen_chat_model(d)

# Load the data for the train and test
# The last param is the num. of rows we take from the begining of the chat
num_of_rows_from_start_of_chat = 6
X, Y = infer_vectors(chat_model, d, num_of_rows_from_start_of_chat)

# Divide into training and test data
x_train, y_train = X[:train_size], Y[:train_size]
x_test, y_test = X[train_size:], Y[train_size:]


# Define the model
model = Sequential()
# model.add(Dense(80, input_dim=100, activation='relu'))
input_layer = Dense(80, input_dim=100, activation='relu')

# one attention layer
# attention_probs = Dense(100, activation='softmax', name='attention_probs')(input_layer)
# attention_mul = merge([input_layer, attention_probs], output_shape=(80,), name='attention_mul', mode='mul')
# model.add(attention_mul)
# model.add(input_layer)
# model.add(Dense(60, activation='relu'))
model.add(Embedding(100, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2)) # recurrent
model.add(Dense(64, activation='softmax', name='attention_probs'))
model.add(Dense(1, activation='sigmoid'))

# Compile and fit
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100, batch_size=32)

# Test the model
scores = model.evaluate(x_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

average number of sents in chat: 14.598977355734112
Epoch 1/100
1000/1000 [==============================] - 10s - loss: 0.6827 - acc: 0.5760    
Epoch 2/100
1000/1000 [==============================] - 9s - loss: 0.6701 - acc: 0.6140     
Epoch 3/100
1000/1000 [==============================] - 9s - loss: 0.6683 - acc: 0.6140     
Epoch 4/100
1000/1000 [==============================] - 10s - loss: 0.6679 - acc: 0.6140    
Epoch 5/100
1000/1000 [==============================] - 9s - loss: 0.6676 - acc: 0.6140     
Epoch 6/100
1000/1000 [==============================] - 9s - loss: 0.6673 - acc: 0.6140     
Epoch 7/100
1000/1000 [==============================] - 9s - loss: 0.6673 - acc: 0.6140     
Epoch 8/100
1000/1000 [==============================] - 10s - loss: 0.6671 - acc: 0.6140    
Epoch 9/100
1000/1000 [==============================] - 10s - loss: 0.6670 - acc: 0.6140    
Epoch 10/100
1000/1000 [==============================] - 10s - loss: 0.6670 - acc: 0.6140    
Epoch 1

RNN network with attention - does not perform well currently

In [210]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, merge, Embedding, recurrent
from keras.models import Model
import keras
import sys
# should change to relative import
sys.path.insert(0, '/home/amirgilad/Documents/git/NLPProj/kerasAttention')
import attention_dense
import attention_lstm

input_dims = x_train.shape
train_size = 1000

# Build doc2Vec model for the chats
d = build_label_dict()
chat_model = gen_chat_model(d)

# Load the data for the train and test
# The last param is the num. of rows we take from the begining of the chat
num_of_rows_from_start_of_chat = 6
X, Y = infer_vectors(chat_model, d, num_of_rows_from_start_of_chat)

# Divide into training and test data
x_train, y_train = X[:train_size], Y[:train_size]
x_test, y_test = X[train_size:], Y[train_size:]



# model = Sequential()
# # model.add(Dense(80, input_dim=100, activation='relu'))
# input_layer = Dense(80, input_dim=100, activation='relu')


RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 100
SENT_HIDDEN_SIZE = 100
BATCH_SIZE = 32
EPOCHS = 40

# Define the model
sentence = Input(shape=(input_dims[1],), dtype='int32')
encoded_sentence = Embedding(100, EMBED_HIDDEN_SIZE)(sentence)
encoded_sentence = layers.Dropout(0.3)(encoded_sentence)
attention_probs = Dense(100, activation='softmax', name='attention_probs')(encoded_sentence)# one attention layer


merged = layers.add([encoded_sentence, attention_probs])
merged = RNN(EMBED_HIDDEN_SIZE)(merged)
merged = layers.Dropout(0.3)(merged)
preds = Dense(1, activation='sigmoid')(merged)

model = Model([sentence], preds)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print('Training')
model.fit([x_train], y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.05)
loss, acc = model.evaluate([x_test], y_test,
                           batch_size=BATCH_SIZE)

print('\nTest loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

average number of sents in chat: 14.598977355734112
Training
Train on 950 samples, validate on 50 samples
Epoch 1/40
950/950 [==============================] - 6s - loss: 0.6687 - acc: 0.6137 - val_loss: 0.7034 - val_acc: 0.5400 loss: 0.6745 - ac
Epoch 2/40
950/950 [==============================] - 6s - loss: 0.6673 - acc: 0.6179 - val_loss: 0.6933 - val_acc: 0.5400
Epoch 3/40
950/950 [==============================] - 7s - loss: 0.6710 - acc: 0.6179 - val_loss: 0.6932 - val_acc: 0.5400
Epoch 4/40
950/950 [==============================] - 7s - loss: 0.6690 - acc: 0.6179 - val_loss: 0.7068 - val_acc: 0.5400
Epoch 5/40
950/950 [==============================] - 7s - loss: 0.6668 - acc: 0.6179 - val_loss: 0.7021 - val_acc: 0.5400
Epoch 6/40
950/950 [==============================] - 7s - loss: 0.6653 - acc: 0.6179 - val_loss: 0.6973 - val_acc: 0.5400
Epoch 7/40
950/950 [==============================] - 7s - loss: 0.6675 - acc: 0.6179 - val_loss: 0.6959 - val_acc: 0.5400
Epoch 8/40
950/

In [25]:
import numpy as np
from keras.layers import merge
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *
import sys
sys.path.insert(0, '/home/amirgilad/Documents/git/NLPProj/kerasAttention')
from attention_utils import get_activations, get_data_recurrent


# Build doc2Vec model for the chats
d = build_label_dict()
chat_model = gen_chat_model(d)

train_size = 1000
batch_size = 50
INPUT_DIM = 100
TIME_STEPS = 6
# if True, the attention vector is shared across the input_dimensions where the attention is applied.
SINGLE_ATTENTION_VECTOR = False
APPLY_ATTENTION_BEFORE_LSTM = False


# Load the data for the train and test
# The last param is the num. of rows we take from the begining of the chat
num_of_rows_from_start_of_chat = TIME_STEPS
X, Y = infer_vectors_every_sentence(chat_model, d, num_of_rows_from_start_of_chat)

# Divide into training and test data
x_train, y_train = X[:train_size], Y[:train_size]
x_test, y_test = X[train_size:], Y[train_size:]

def get_data_recurrent(n, time_steps, input_dim, attention_column=10):
    """
    Data generation. x is purely random except that it's first value equals the target y.
    In practice, the network should learn that the target = x[attention_column].
    Therefore, most of its attention should be focused on the value addressed by attention_column.
    :param n: the number of samples to retrieve.
    :param time_steps: the number of time steps of your series.
    :param input_dim: the number of dimensions of each element in the series.
    :param attention_column: the column linked to the target. Everything else is purely random.
    :return: x: model inputs, y: model targets
    """
#     x = np.random.standard_normal(size=(n, time_steps, input_dim))
    
    sampler = np.random.choice(train_size, batch_size)
    x = x_train[sampler, :num_of_rows_from_start_of_chat, :input_dim]
#     y = np.random.randint(low=0, high=2, size=(n, 1))
    y = y_train[sampler]
#     x[:, attention_column, :] = np.tile(y[:], (1, input_dim))
    return x, y

def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
#     print(inputs.shape)
    input_dim = INPUT_DIM#int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a)
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
#     printoutput_attention_mul.shape
    return output_attention_mul


def model_attention_applied_after_lstm():
    inputs = Input(shape=(TIME_STEPS, INPUT_DIM,))
    lstm_units = 32
    lstm_out = LSTM(lstm_units, return_sequences=True)(inputs)
    attention_mul = attention_3d_block(lstm_out)
    attention_mul = Flatten()(attention_mul)
    output = Dense(1, activation='sigmoid')(attention_mul)
    model = Model(input=[inputs], output=output)
    return model


def model_attention_applied_before_lstm():
    inputs = Input(shape=(TIME_STEPS, INPUT_DIM,))
    attention_mul = attention_3d_block(inputs)
    lstm_units = 32
    attention_mul = LSTM(lstm_units, return_sequences=False)(attention_mul)
    output = Dense(1, activation='sigmoid')(attention_mul)
    model = Model(input=[inputs], output=output)
    return model

N = 1369
# N = 300 -> too few = no training
inputs_1, outputs = get_data_recurrent(N, TIME_STEPS, INPUT_DIM)

# if APPLY_ATTENTION_BEFORE_LSTM:
#     m = model_attention_applied_after_lstm()
# else:
m = model_attention_applied_before_lstm()

m.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(m.summary())

m.fit([inputs_1], outputs, epochs=1, batch_size=64, validation_split=0.1)

attention_vectors = []
for i in range(300):
    testing_inputs_1, testing_outputs = get_data_recurrent(batch_size, TIME_STEPS, INPUT_DIM)
    attention_vector = np.mean(get_activations(m,
                                               testing_inputs_1,
                                               print_shape_only=True,
                                               layer_name='attention_vec')[0], axis=2).squeeze()
    print('attention =', attention_vector)
    print(np.sum(attention_vector) - 1.0)
#     assert (np.sum(attention_vector) - 1.0) < 1e-5
    attention_vectors.append(attention_vector)

attention_vector_final = np.mean(np.array(attention_vectors), axis=0)


loss, acc = m.evaluate([x_test], y_test,
                           batch_size=batch_size)

print('\nTest loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

# plot part.
# import matplotlib.pyplot as plt
# import pandas as pd

# pd.DataFrame(attention_vector_final, columns=['attention (%)']).plot(kind='bar',
#                                                                      title='Attention Mechanism as '
#                                                                            'a function of input'
#                                                                            ' dimensions.')
# plt.show()

average number of sents in chat: 14.598977355734112


/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:64: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python3.4/dist-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:86: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[permute_3..., outputs=sigmoid.0)`


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_11 (InputLayer)            (None, 6, 100)        0                                            
____________________________________________________________________________________________________
permute_10 (Permute)             (None, 100, 6)        0           input_11[0][0]                   
____________________________________________________________________________________________________
reshape_8 (Reshape)              (None, 100, 6)        0           permute_10[0][0]                 
____________________________________________________________________________________________________
dense_14 (Dense)                 (None, 100, 6)        42          reshape_8[0][0]                  
___________________________________________________________________________________________

(50, 6, 100)
attention = [[ 0.16720635  0.16657749  0.1670389   0.16655587  0.16622449  0.16639687]
 [ 0.16719647  0.16644309  0.16654545  0.16667205  0.16629142  0.16685152]
 [ 0.16719702  0.16652983  0.16655849  0.16587685  0.16626751  0.16757025]
 [ 0.16698544  0.16635118  0.16660017  0.16694395  0.16667034  0.16644898]
 [ 0.16877231  0.16541973  0.16762619  0.16579594  0.16635041  0.16603538]
 [ 0.16708836  0.166501    0.16667978  0.16643368  0.16678551  0.16651168]
 [ 0.1671039   0.16627221  0.16668478  0.16694443  0.16657454  0.16642018]
 [ 0.16770436  0.16630058  0.1671374   0.16523103  0.16615447  0.16747209]
 [ 0.1668022   0.16652279  0.16624574  0.16714975  0.16687727  0.16640222]
 [ 0.16698427  0.16676342  0.16713062  0.16636871  0.16613339  0.16661963]
 [ 0.16671734  0.16689137  0.16699396  0.16642654  0.16661929  0.16635153]
 [ 0.16693717  0.16624759  0.16621149  0.16663794  0.16697706  0.16698879]
 [ 0.16693571  0.16671024  0.16686782  0.16639489  0.16657682  0.16651444]


(50, 6, 100)
attention = [[ 0.16720064  0.16638948  0.16652675  0.1664422   0.16654602  0.16689491]
 [ 0.16692939  0.16660689  0.16669929  0.16660005  0.16625327  0.16691098]
 [ 0.16738982  0.16611439  0.16622898  0.16688164  0.16615404  0.1672311 ]
 [ 0.167881    0.16573876  0.1663945   0.16646291  0.16728938  0.16623341]
 [ 0.16738692  0.16624342  0.16683707  0.16653469  0.16626965  0.16672824]
 [ 0.16635782  0.16703185  0.1669738   0.16721591  0.166537    0.16588363]
 [ 0.16777636  0.16572721  0.16637243  0.16681431  0.16638659  0.16692309]
 [ 0.16728733  0.16639692  0.16695942  0.16707449  0.16637385  0.1659079 ]
 [ 0.1674435   0.16620842  0.16653115  0.16639112  0.16665855  0.1667673 ]
 [ 0.16759293  0.16612028  0.16693041  0.16604544  0.16637549  0.16693552]
 [ 0.1676804   0.16595767  0.16706541  0.16656609  0.16597107  0.1667593 ]
 [ 0.16655068  0.16671444  0.16649103  0.1667973   0.16705431  0.16639231]
 [ 0.16734022  0.16669799  0.16733015  0.16586     0.16588306  0.16688854]


...

(50, 6, 100)
attention = [[ 0.16708462  0.16589993  0.16582477  0.16693252  0.16779704  0.16646113]
 [ 0.16718982  0.16618954  0.16632359  0.16677582  0.1671394   0.16638178]
 [ 0.16635466  0.16659056  0.16579801  0.16770913  0.16767599  0.16587168]
 [ 0.1666297   0.16688988  0.16658483  0.16698864  0.16643502  0.16647187]
 [ 0.16738456  0.16627108  0.16663577  0.16611643  0.16647324  0.16711883]
 [ 0.16822864  0.16556726  0.16662169  0.16629526  0.1653285   0.16795866]
 [ 0.16696677  0.16658457  0.1666408   0.16691169  0.16649418  0.16640204]
 [ 0.16699204  0.16638847  0.16646767  0.16700907  0.16634017  0.16680261]
 [ 0.16791268  0.16567141  0.16643327  0.16661867  0.16693124  0.16643274]
 [ 0.16693768  0.1667155   0.16699579  0.16633686  0.16640733  0.16660689]
 [ 0.16696011  0.166723    0.1666608   0.16697222  0.16598645  0.16669746]
 [ 0.16717924  0.16631514  0.16657208  0.16629171  0.16633411  0.16730772]
 [ 0.16652216  0.16654977  0.16628376  0.16710433  0.16732599  0.166214  ]


(50, 6, 100)
attention = [[ 0.16686359  0.16654241  0.16646026  0.16668777  0.16674055  0.1667054 ]
 [ 0.16685303  0.16665886  0.16650586  0.16641174  0.16669615  0.16687432]
 [ 0.16723107  0.16612332  0.16648538  0.16611801  0.16683768  0.16720457]
 [ 0.16739587  0.16568738  0.16565037  0.16701932  0.16820265  0.16604444]
 [ 0.16710947  0.1664218   0.16686778  0.16665266  0.16619903  0.1667493 ]
 [ 0.1663539   0.16729945  0.16684812  0.1668324   0.16635369  0.16631261]
 [ 0.16686785  0.16683128  0.1674666   0.16729854  0.16547996  0.16605572]
 [ 0.16669619  0.16710426  0.16670395  0.16691421  0.16634007  0.16624132]
 [ 0.16686359  0.16654241  0.16646026  0.16668777  0.16674055  0.1667054 ]
 [ 0.16648458  0.16704364  0.16714248  0.16668254  0.16612123  0.16652545]
 [ 0.16727322  0.16637026  0.16695838  0.16622509  0.16644089  0.16673224]
 [ 0.1667202   0.16704601  0.16735858  0.16682725  0.16619323  0.16585478]
 [ 0.16713491  0.16608123  0.16597436  0.16730803  0.16730255  0.16619898]


(50, 6, 100)
attention = [[ 0.16639119  0.16658771  0.16563547  0.16690418  0.16797556  0.16650601]
 [ 0.16694894  0.16633976  0.16642644  0.16687146  0.16636227  0.16705114]
 [ 0.16686386  0.16680808  0.16729827  0.16674533  0.16597448  0.16631006]
 [ 0.16695222  0.16685599  0.16680574  0.16688034  0.16571704  0.16678867]
 [ 0.16750807  0.16623543  0.16682251  0.16582538  0.16660126  0.16700739]
 [ 0.16716765  0.16652276  0.16659741  0.16614605  0.16660643  0.16695972]
 [ 0.16696982  0.1662481   0.16672741  0.16781689  0.16670161  0.1655362 ]
 [ 0.16692589  0.16670504  0.16661993  0.16636537  0.16631296  0.16707073]
 [ 0.16704252  0.16655377  0.16675971  0.16655274  0.16662087  0.16647047]
 [ 0.16762732  0.16584314  0.16675766  0.16671963  0.16673055  0.16632175]
 [ 0.16702127  0.16616841  0.16591267  0.16697183  0.16667946  0.1672464 ]
 [ 0.16705726  0.16668682  0.16688211  0.16648266  0.16598496  0.16690621]
 [ 0.16747609  0.16632971  0.16710953  0.16615805  0.16649641  0.16643018]


(50, 6, 100)
attention = [[ 0.16713789  0.16622719  0.16649823  0.16678004  0.16673945  0.16661721]
 [ 0.16746794  0.16592176  0.16626671  0.16632895  0.16722888  0.16678579]
 [ 0.16760628  0.16597645  0.16656291  0.16665298  0.16676119  0.16644029]
 [ 0.1669554   0.16680133  0.16690744  0.16659839  0.16660658  0.16613089]
 [ 0.16777636  0.16572721  0.16637243  0.16681431  0.16638659  0.16692309]
 [ 0.16713789  0.16622719  0.16649823  0.16678004  0.16673945  0.16661721]
 [ 0.16730356  0.16631377  0.166601    0.16627583  0.16644594  0.16705991]
 [ 0.16714153  0.16631383  0.1661503   0.16685338  0.16676612  0.16677488]
 [ 0.1669798   0.16622038  0.16626659  0.16635507  0.16747421  0.16670388]
 [ 0.16687672  0.16678256  0.16703197  0.16655348  0.16646011  0.16629511]
 [ 0.16700913  0.16629909  0.16629496  0.16659121  0.16677603  0.16702959]
 [ 0.16820215  0.16550337  0.16675811  0.16596967  0.16640574  0.16716091]
 [ 0.16678531  0.16644442  0.16596405  0.1669106   0.16687241  0.16702326]


 50/369 [===>..........................] - ETA: 0s
Test loss / test accuracy = 0.6927 / 0.6070
